In [1]:
%load_ext autoreload
%autoreload 2
from synthpop.recipes.starter6 import Starter
starter_ver='6'

from output_verify import output_verify
from synthpop.synthesizer import synthesize_all, enable_logging 
import pandas as pd
import os,time
import pickle

outputfolder='/home/da/tools/synthpop/demos/outputs/'

# Synthesize by county 

In [ ]:
#for whole county
time0=time.time()
results=[]
def synthesize_counties(counties):
    for county in counties:
        for phase in range(3):
            pathout=outputfolder+county+"_starter%s_"%(starter_ver)
            starter = Starter(os.environ["CENSUS"], "MI", county, phase=phase)
            households, persons, fit, marginals=synthesize_all(starter)

            households.index.name='hh_id' #result[0] is hh
            households['serialno'] = households['serialno'].map(lambda x: '%f' % x)
            persons['serialno'] = persons['serialno'].map(lambda x: '%f' % x)
            #results[0].to_csv(outputfolder+county+"_starter%s_"%('3')+"HHs.csv",float_format='%.f') #float_format, not use scentific, pplies to all fp numbers 
            #could use results['serialno'] = results['serialno'].map(lambda x: '%f' % x) to change output format for serialno only
            households.to_csv(pathout+'_households.csv')
            persons.to_csv(pathout+'_persons.csv')

            df_verify=output_verify(households,persons,marginals)
            df_verify[0].to_csv(pathout+"_verify_households.csv")
            df_verify[1].to_csv(pathout+"_verify_persons.csv")
            df_verify[2].to_csv(pathout+"_verify_hh_size.csv")
            if phase == 0:
                df_verify[3].to_csv("hh7_persons_drawing.csv")
            if phase == 1:
                df_ttlpersons=pd.DataFrame(data=[totalpersons],columns=['hh_size_person_factor'])
                df_verify[4].to_csv("hh_size_person_factor.csv")
            if phase == 2:
                os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
                os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")

            with open(pathout+'_fit_stats.pkl', 'wb') as f:
                pickle.dump(fit, f)
            print time.time()-time0
    return households, persons, fit, marginals
#synthesize_counties(["Livingston County","Oakland County","Washtenaw County","Wayne County","Monroe County","Macomb County"])
res=synthesize_counties(["Macomb County"])


Synthesizing at geog level: 'block_group' (number of geographies is 627)
Synthesizing geog id:
state              26
county            099
tract          206700
block group         1
dtype: object
Drawing 749 households
Synthesizing geog id:
state              26
county            099
tract          206700
block group         2
dtype: object
Drawing 783 households
Synthesizing geog id:
state              26
county            099
tract          210000
block group         1
dtype: object
Drawing 722 households
Synthesizing geog id:
state              26
county            099
tract          210000
block group         2
dtype: object
Drawing 737 households
Synthesizing geog id:
state              26
county            099
tract          211000
block group         1
dtype: object
Drawing 722 households
Synthesizing geog id:
state              26
county            099
tract          211000
block group         2
dtype: object
Drawing 467 households
Synthesizing geog id:
state              26
c

# Synthesize by selected BGs

In [ ]:
###for selected BGs
#enable_logging()
time0=time.time()
geoname="problembgs.csv"
pathout=outputfolder+geoname+"_starter%s_"%(starter_ver)

dfbg=pd.read_csv(geoname,index_col=None,dtype=str) #make sure csv column in text not number, so read as'099' not 99
indd=[ind for i, ind in dfbg.iterrows()]
for phase in range(3):
    print "\nphase = ", phase, "\n"
    starter = Starter(os.environ["CENSUS"], "MI", 'Macomb County', phase=phase)
    households, persons, fit, marginals = synthesize_all(starter, indexes=indd)
    print fit
    #HHs and Persons output
    households.to_csv(pathout+'_households.csv')
    persons.to_csv(pathout+'_persons.csv')

    df_verify=output_verify(households,persons,marginals)
    df_verify[0].to_csv(pathout+"_verify_households.csv")
    df_verify[1].to_csv(pathout+"_verify_persons.csv")
    df_verify[2].to_csv(pathout+"_verify_hh_size.csv")
    if phase == 0:
        df_verify[3].to_csv("hh7_persons_drawing.csv")
    if phase == 1:
        df_verify[4].to_csv("hh_size_person_factor.csv")
    if phase == 2:
        os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
        os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")
print time.time()-time0